# Emerging Technologies

In [53]:
import random

## Problem 1: Generating Random Boolean Functions

The **Deutsch-Jozsa Algorithm** needs test cases to prove that it works. The function `random_constant_balanced()` is a test data generator, that creates random functions that the algorithm can practice on.

The reason why the function is made to generate the test cases is that doing it manually would be very tedious. An example would be if you have a sorting algorithm, you would need random lists to test it on! This is the equivalent to that but for Deutsch-Jozsa.

In [ ]:
def random_constant_balanced():

    choice = random.choice(["constant", "balanced"])
    
    if choice == "constant":
        value = random.choice([True, False])
        
        def my_function(x1, x2, x3, x4):
            return value
        
        return my_function
    
    else:

        all_inputs = []

        for i in range(16):

            # Convert number to binary (4 bits)
            x1 = bool((i >> 3) & 1)  # bit 3
            x2 = bool((i >> 2) & 1)  # bit 2
            x3 = bool((i >> 1) & 1)  # bit 1
            x4 = bool(i & 1)         # bit 0
            all_inputs.append((x1, x2, x3, x4))
        
        true_inputs = random.sample(all_inputs, 8)
        
        def my_function(x1, x2, x3, x4):
            return (x1, x2, x3, x4) in true_inputs
        
        return my_function

### Test Case

In [55]:
print("Generating 5 random functions:\n")

for i in range(5):
    f = random_constant_balanced()
    
    # Count True outputs
    true_count = sum(f(bool((j>>3)&1), bool((j>>2)&1), 
                        bool((j>>1)&1), bool(j&1)) 
                     for j in range(16))
    
    if true_count == 8:
        func_type = "Balanced"
    elif true_count == 0:
        func_type = "Constant (False)"
    else:
        func_type = "Constant (True)"
    
    print(f"Function {i+1}: {func_type} - {true_count} True outputs")

Generating 5 random functions:

Function 1: Balanced - 8 True outputs
Function 2: Constant (False) - 0 True outputs
Function 3: Constant (True) - 16 True outputs
Function 4: Constant (False) - 0 True outputs
Function 5: Constant (False) - 0 True outputs


## Problem 2: Classical Testing for Function Type

## Problem 3: Quantum Oracles

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm